In [1]:
# pip install yfinance

In [2]:
# pip install cchardet

In [3]:
# pip install yahoo_fin

In [4]:
import pandas as pd
import numpy as np
# from stocknews import StockNews
from datetime import date
import datetime
import yfinance as yf
from tqdm import tqdm
import statistics
import cchardet
import yahoo_fin.stock_info as si
import gc
from functools import reduce
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt', sep = '|')
df = df[(df.Symbol.notnull()) & (df.ETF == 'N')]
df = df[df['Round Lot Size'] == 100]
print(df.shape)

(7516, 12)


In [6]:
# symbols = symbols[:100]

In [7]:
# data = data[data.Symbol.isin(symbols)]

In [8]:
df.head()

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
4,Y,AAC.U,"Ares Acquisition Corporation Units, each consi...",N,,N,100.0,N,NaN,AAC.U,AAC=,N
5,Y,AACG,ATA Creativity Global - American Depositary Sh...,Q,G,N,100.0,N,N,NaN,AACG,N
6,Y,AACQ,Artius Acquisition Inc. - Class A Common Stock,Q,S,N,100.0,N,N,NaN,AACQ,N


### Universe

In [9]:
yahoo = pd.read_pickle('yahoo_01-02.pkl')

yahoo_Close = yahoo[yahoo.level_0 == 'Close']
yahoo_Close = yahoo_Close.iloc[:,1:]
close_cols = ['Close_' + col.strftime('%Y-%m-%d') for col in yahoo_Close.columns if col != 'level_1']
yahoo_Close.columns = ['Symbol'] + close_cols

yahoo_Volume = yahoo[yahoo.level_0 == 'Volume']
yahoo_Volume = yahoo_Volume.iloc[:,1:]
close_cols = ['Volume_' + col.strftime('%Y-%m-%d') for col in yahoo_Volume.columns if col != 'level_1']
yahoo_Volume.columns = ['Symbol'] + close_cols

df = df.merge(yahoo_Close, on = 'Symbol', how = 'left')
df = df.merge(yahoo_Volume, on = 'Symbol', how = 'left')

df = df.dropna(axis = 1, how = 'all')
df = df.dropna(axis = 0, how = 'all')

close_cols = [col for col in df.columns if 'Close' in col]
vol_cols = [col for col in df.columns if 'Volume' in col]

df['average_close'] = df[close_cols[:1]].mean(axis = 1)
df['average_vol'] = df[vol_cols[:1]].mean(axis = 1)

# df = df[df.average_close <= 1]
# data = data[data.average_close >= 2]

# data = data.sort_values('average_vol', ascending = False).head(50).reset_index(drop = True)
df.shape

(7516, 62)

In [10]:
final_symbols = df.Symbol.tolist()
# final_symbols.extend(['AAPL','TSLA','AMZN'])
len(final_symbols)

7516

## Get Stats_Valuation Data

In [13]:
dow_list = final_symbols
 
# Get data in the current column for each stock's valuation table
dow_stats = {}
for ticker in tqdm(dow_list):
    try:
        temp = si.get_stats_valuation(ticker)
        temp = temp.iloc[:,:2]
        temp.columns = ["Attribute", "Recent"]

        dow_stats[ticker] = temp
    except:
        continue
 
 
# combine all the stats valuation tables into a single data frame
combined_stats = pd.concat(dow_stats)
combined_stats = combined_stats.reset_index()
 
del combined_stats["level_1"]
 
# update column names
combined_stats.columns = ["Ticker", "Attribute", "Recent"]
combined_stats['Values'] = (combined_stats.Recent.replace(r'[kMBT]+$', '', regex=True).astype(np.float64) * combined_stats.Recent.str.extract(r'[\d\.]+([kMBT]+)', expand=False).fillna(1).replace(['k','M','B','T'], [10**3, 10**6, 10**9, 10**12]).astype(np.uint64))
combined_stats = combined_stats.drop('Recent', axis = 1)

100%|██████████| 7516/7516 [2:56:03<00:00,  1.41s/it]  


In [14]:
final_stats = combined_stats.pivot(index = 'Ticker', columns = 'Attribute', values = 'Values')

In [15]:
cols_to_keep = ['Enterprise Value/EBITDA 6',
                'Enterprise Value/Revenue 3',
#                 'Forward P/E 1',
#                 'PEG Ratio (5 yr expected) 1',
                'Price/Book (mrq)',
                'Price/Sales (ttm)',
#                 'Trailing P/E'
               ]

final_stats = final_stats[cols_to_keep]

In [16]:
final_stats

Attribute,Enterprise Value/EBITDA 6,Enterprise Value/Revenue 3,Price/Book (mrq),Price/Sales (ttm)
Ticker,,,,
A,31.06,7.40,8.27,7.32
AA,4.92,0.52,1.19,0.42
AACG,-1.33,1.10,4.64,7.19
AACQ,NaN,NaN,NaN,NaN
AACQU,NaN,NaN,NaN,NaN
...,...,...,...,...
AEPPL,NaN,NaN,NaN,NaN
AEPPZ,NaN,NaN,NaN,NaN
AER,11.40,7.35,0.61,1.18


## Get Valuation Extra Data

In [17]:
dow_extra_stats = {}
for ticker in tqdm(dow_list):
    try:
        dow_extra_stats[ticker] = si.get_stats(ticker)
    except:
        continue
     
combined_extra_stats = pd.concat(dow_extra_stats)
combined_extra_stats = combined_extra_stats.reset_index()
combined_extra_stats = combined_extra_stats.replace(',','', regex=True)
combined_extra_stats = combined_extra_stats.replace('%','P', regex=True)

100%|██████████| 7516/7516 [2:55:23<00:00,  1.40s/it]  


In [18]:
date_cols = [col for col in combined_extra_stats.Attribute.tolist() if 'Date' in col]
date_cols
combined_extra_stats = combined_extra_stats[~combined_extra_stats.Attribute.isin(['Fiscal Year Ends','Most Recent Quarter (mrq)','Last Split Factor 2'] + date_cols)]

In [19]:
combined_extra_stats['Values'] = (combined_extra_stats.Value.replace(r'[kMBTP]+$', '', regex=True).astype(np.float64) * combined_extra_stats.Value.str.extract(r'[\d\.]+([kMBTP]+)', expand=False).fillna(1).replace(['k','M','B','T','P'], [10**3, 10**6, 10**9, 10**12, 10**-2]).astype(np.float64))
combined_extra_stats = combined_extra_stats.drop(['Value','level_1'], axis = 1)

In [20]:
final_extra_stats = combined_extra_stats.pivot(index = 'level_0', columns = 'Attribute', values = 'Values')

In [21]:
equity_cols = ['200-Day Moving Average 3',
               '5 Year Average Dividend Yield 4',
               '50-Day Moving Average 3',
               '52 Week High 3',
               '52 Week Low 3',
               '52-Week Change 3',
               'Avg Vol (10 day) 3',
               'Avg Vol (3 month) 3',
               'Forward Annual Dividend Rate 4',
               'Forward Annual Dividend Yield 4',
               'Float',
               'Net Income Avi to Common (ttm)',
               'S&P500 52-Week Change 3',
               'Shares Outstanding 5',
               'Shares Short (Jan 28 2021) 4',
               'Shares Short (prior month Dec 30 2020) 4',
              'Trailing Annual Dividend Rate 3',
              'Trailing Annual Dividend Yield 3']

non_equity_cols = [col for col in final_extra_stats.columns if col not in equity_cols]
final_extra_stats = final_extra_stats[non_equity_cols]

In [22]:
final_extra_stats.head(60)

Attribute,5y Average Return,Average for Category,Beta (5Y Monthly),Book Value Per Share (mrq),Current Ratio (mrq),Diluted EPS (ttm),EBITDA,Expense Ratio (net),Gross Profit (ttm),Holdings Turnover,Last Dividend,Levered Free Cash Flow (ttm),NAV,Net Assets,Operating Cash Flow (ttm),Operating Margin (ttm),P Held by Insiders 1,P Held by Institutions 1,PE Ratio (TTM),Payout Ratio 4,Profit Margin,Quarterly Earnings Growth (yoy),Quarterly Revenue Growth (yoy),Return on Assets (ttm),Return on Equity (ttm),Revenue (ttm),Revenue Per Share (ttm),Shares Short (Aug 30 2017) 4,Shares Short (Aug 30 2018) 4,Shares Short (Dec 30 2020) 4,Shares Short (Jan 14 2020) 4,Shares Short (Jan 14 2021) 4,Shares Short (Jun 11 2001) 4,Shares Short (prior month ) 4,Shares Short (prior month Apr 08 2001) 4,Shares Short (prior month Aug 29 2019) 4,Shares Short (prior month Dec 14 2020) 4,Shares Short (prior month Jul 30 2017) 4,Shares Short (prior month Jul 30 2018) 4,Shares Short (prior month May 14 2020) 4,Shares Short (prior month Nov 12 2020) 4,Shares Short (prior month Nov 29 2020) 4,Shares Short 4,Short P of Float (Aug 30 2017) 4,Short P of Float (Aug 30 2018) 4,Short P of Float (Dec 30 2020) 4,Short P of Float (Jan 14 2020) 4,Short P of Float (Jan 14 2021) 4,Short P of Float (Jan 28 2021) 4,Short P of Float (Jun 11 2001) 4,Short P of Float 4,Short P of Shares Outstanding (Aug 30 2017) 4,Short P of Shares Outstanding (Aug 30 2018) 4,Short P of Shares Outstanding (Dec 30 2020) 4,Short P of Shares Outstanding (Jan 14 2020) 4,Short P of Shares Outstanding (Jan 14 2021) 4,Short P of Shares Outstanding (Jan 28 2021) 4,Short P of Shares Outstanding (Jun 11 2001) 4,Short P of Shares Outstanding 4,Short Ratio (Aug 30 2017) 4,Short Ratio (Aug 30 2018) 4,Short Ratio (Dec 30 2020) 4,Short Ratio (Jan 14 2020) 4,Short Ratio (Jan 14 2021) 4,Short Ratio (Jan 28 2021) 4,Short Ratio (Jun 11 2001) 4,Short Ratio 4,Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),YTD Daily Total Return,Yield
level_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A,NaN,NaN,NaN,15.75,2.07,2.600,1.460000e+09,NaN,2.840000e+09,NaN,NaN,8.800000e+08,NaN,NaN,1.220000e+09,0.2092,0.0032,0.9007,NaN,0.2823,0.1465,0.462,0.141,0.0754,0.1678,5.530000e+09,17.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.37,NaN,NaN,1.330000e+09,4.36,2.500000e+09,52.02,NaN,NaN
AA,NaN,NaN,NaN,17.67,1.64,-0.920,1.080000e+09,NaN,1.320000e+09,NaN,NaN,4.991200e+08,NaN,NaN,3.940000e+08,0.0464,0.0055,0.7692,NaN,0.0000,-0.0183,NaN,-0.018,0.0183,-0.0026,9.290000e+09,49.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.82,NaN,NaN,1.610000e+09,8.64,2.460000e+09,49.39,NaN,NaN
AACG,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,3.586000e+07,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0004,NaN,0.0000,-0.6697,NaN,0.039,-0.1053,-0.3377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN,NaN,12.12,NaN,NaN
AACQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACQU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACQW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Merge Data

In [138]:
comb_df = pd.concat([final_stats, final_extra_stats], axis = 1)

In [139]:
# na_cols = ['Quarterly Earnings Growth (yoy)','Shares Short (prior month ) 4','Shares Short 4','Short P of Float 4','Short P of Shares Outstanding 4','Short Ratio 4']
# comb_df.drop(na_cols, axis = 1, inplace = True)

In [140]:
non_na_cols = comb_df.isnull().sum()[comb_df.isnull().sum() < 3000].sort_values(ascending = False).index
comb_df = comb_df[non_na_cols]
comb_df = comb_df.dropna()
comb_df.shape

(2283, 24)

In [141]:
comb_df.head(20)

Attribute,Levered Free Cash Flow (ttm),EBITDA,Operating Cash Flow (ttm),Total Debt/Equity (mrq),Short P of Float (Jan 28 2021) 4,Revenue Per Share (ttm),Revenue (ttm),Payout Ratio 4,Diluted EPS (ttm),Book Value Per Share (mrq),Return on Equity (ttm),Total Debt (mrq),Quarterly Revenue Growth (yoy),Current Ratio (mrq),Return on Assets (ttm),Gross Profit (ttm),Total Cash Per Share (mrq),Total Cash (mrq),Short P of Shares Outstanding (Jan 28 2021) 4,P Held by Institutions 1,P Held by Insiders 1,Short Ratio (Jan 28 2021) 4,Profit Margin,Operating Margin (ttm)
A,8.800000e+08,1.460000e+09,1.220000e+09,52.02,0.0075,17.95,5.530000e+09,0.2823,2.600,15.75,0.1678,2.500000e+09,0.141,2.07,0.0754,2.840000e+09,4.36,1.330000e+09,0.0074,0.9007,0.0032,1.37,0.1465,0.2092
AA,4.991200e+08,1.080000e+09,3.940000e+08,49.39,0.0671,49.96,9.290000e+09,0.0000,-0.920,17.67,-0.0026,2.460000e+09,-0.018,1.64,0.0183,1.320000e+09,8.64,1.610000e+09,0.0599,0.7692,0.0055,1.82,-0.0183,0.0464
AAME,8.440000e+06,5.070000e+06,4.860000e+06,30.44,0.0004,9.18,1.877000e+08,0.0000,0.058,6.27,0.0130,3.897000e+07,-0.031,1.25,0.0063,1.348000e+07,0.44,8.940000e+06,0.0001,0.0552,0.8003,0.02,0.0085,0.0207
AAOI,-4.835000e+07,-2.335000e+07,-3.307000e+07,59.95,0.2139,11.01,2.309600e+08,0.0000,-3.840,11.59,-0.2835,1.589600e+08,0.662,1.74,-0.0608,4.620000e+07,2.04,4.677000e+07,0.1554,0.4895,0.0536,3.57,-0.3483,-0.2067
AAON,3.754000e+07,1.254400e+08,1.243400e+08,0.46,0.0969,9.98,5.204200e+08,0.2381,1.470,6.60,0.2479,1.600000e+06,0.187,3.67,0.1599,1.194200e+08,1.35,7.060000e+07,0.0646,0.7285,0.2091,15.77,0.1487,0.1947
AAP,6.591700e+08,1.110000e+09,9.696900e+08,98.61,0.0678,147.01,1.011000e+10,0.1401,7.140,52.67,0.1387,3.510000e+09,0.120,1.32,0.0466,4.480000e+09,12.35,8.349900e+08,0.0577,1.0228,0.0037,3.69,0.0488,0.0851
AAPL,6.689000e+10,8.516000e+10,8.892000e+10,169.19,0.0055,17.13,2.941400e+11,0.2177,3.690,3.94,0.8209,1.120400e+11,0.214,1.16,0.1336,1.049600e+11,4.58,7.683000e+10,0.0055,0.5966,0.0007,0.79,0.2174,0.2525
AAT,5.903000e+07,1.929400e+08,1.269800e+08,114.99,0.0309,5.72,3.420900e+08,2.1739,0.460,21.02,0.0279,1.440000e+09,-0.173,3.63,0.0197,2.254800e+08,2.27,1.373300e+08,0.0175,0.9710,0.0151,4.61,0.0808,0.2589
AAWW,6.182700e+08,8.346700e+08,1.010000e+09,123.62,0.1466,121.60,3.210000e+09,0.0000,13.500,82.19,0.1778,2.800000e+09,0.248,1.06,0.0555,1.140000e+09,30.73,8.455900e+08,0.1080,1.0014,0.0158,6.60,0.1122,0.1577
ABM,3.833900e+08,3.725000e+08,4.575000e+08,60.43,0.0236,89.50,5.990000e+09,248.9906,0.000,22.48,0.0001,9.067000e+08,-0.099,1.46,0.0462,8.985000e+08,5.91,3.942000e+08,0.0172,0.9556,0.0105,3.25,0.0000,0.0461


In [143]:
# comb_df.to_pickle('combined_data3.pkl')

## Removing nulls and infs

## Scaling Data and Getting Nearest Neighbors

In [144]:
comb_df = comb_df.apply(lambda x: x.fillna(x.mean()))
comb_df = comb_df.apply(lambda x: (x - x.mean()) / x.std())

In [171]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [172]:
neigh = NearestNeighbors(n_neighbors=3, radius=0.4)
neigh.fit(comb_df.values)

NearestNeighbors(n_neighbors=3, radius=0.4)

In [173]:
neighbors = pd.DataFrame(neigh.kneighbors(comb_df.values, return_distance = False))

In [174]:
res = pd.concat([comb_df.reset_index(drop = True),neighbors.reset_index(drop = True)], axis = 1)
res['Ticker'] =comb_df.index

In [177]:
#Closest Tickers
for i in tqdm(range(res.shape[0])):
    res.loc[i,f'Dist_1'] = res.Ticker[res[1][i]]
    res.loc[i,f'Dist_2'] = res.Ticker[res[2][i]]
#     res.loc[i,f'Dist_3'] = res.Ticker[res[3][i]]
#     res.loc[i,f'Dist_4'] = res.Ticker[res[4][i]]

100%|██████████| 2283/2283 [00:01<00:00, 1328.42it/s]


In [116]:
latest = yf.download(tickers=final_symbols, period='1d', interval='5m')
latest = latest.T.filter(like = 'Close', axis = 0).reset_index()
latest = latest[latest.level_0 == 'Close'].set_index('level_1')
latest = latest.T.tail(1).reset_index(drop = True).T.to_dict()[0]

[*********************100%***********************]  7516 of 7516 completed

919 Failed downloads:
- KEY$I: No data found, symbol may be delisted
- DFVS: No data found for this date range, symbol may be delisted
- APO$B: No data found, symbol may be delisted
- NLY$I: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- NS$B: No data found, symbol may be delisted
- GLP$A: No data found, symbol may be delisted
- PRIF$F: No data found, symbol may be delisted
- CCX.U: No data found, symbol may be delisted
- BAC$M: No data found, symbol may be delisted
- GLOP$A: No data found, symbol may be delisted
- AHH$A: No data found, symbol may be delisted
- STT$D: No data found, symbol may be delisted
- SYF$A: No data found, symbol may be delisted
- EPR$G: No data found, symbol may be delisted
- PCG$E: No data found, symbol may be delisted
- RPT$D: No data found, symbol may be delisted
- NCZ$A: No data found, symbol may be delisted
- PEB$D: No data found, symbol may b

In [59]:
# import joblib
# with open('latest_prices', 'wb') as filename:
#     joblib.dump(latest, filename)

In [178]:
res['Ticker_price'] = res['Ticker'].map(latest)
res['ret_1'] = res['Dist_1'].map(latest)
res['ret_2'] = res['Dist_2'].map(latest)
# res['ret_3'] = res['Dist_3'].map(latest)
# res['ret_4'] = res['Dist_4'].map(latest)

In [179]:
# res['ret_mean'] = res[['ret_1','ret_2','ret_3','ret_4']].mean(axis = 1, skipna = False)
res['ret_mean'] = res[['ret_1','ret_2']].mean(axis = 1, skipna = False)
res['Dist_from_mean'] = (res['ret_mean'] - res['Ticker_price']) / res['Ticker_price']

In [180]:
latest['MSFT']

241.02999877929688

In [181]:
res.dropna().sort_values('Dist_from_mean', ascending = False).head(50)

,Levered Free Cash Flow (ttm),EBITDA,Operating Cash Flow (ttm),Total Debt/Equity (mrq),Short P of Float (Jan 28 2021) 4,Revenue Per Share (ttm),Revenue (ttm),Payout Ratio 4,Diluted EPS (ttm),Book Value Per Share (mrq),Return on Equity (ttm),Total Debt (mrq),Quarterly Revenue Growth (yoy),Current Ratio (mrq),Return on Assets (ttm),Gross Profit (ttm),Total Cash Per Share (mrq),Total Cash (mrq),Short P of Shares Outstanding (Jan 28 2021) 4,P Held by Institutions 1,P Held by Insiders 1,Short Ratio (Jan 28 2021) 4,Profit Margin,Operating Margin (ttm),0,1,2,Ticker,Dist_1,Dist_2,Ticker_price,ret_1,ret_2,ret_mean,Dist_from_mean
1584,-0.180872,-0.219385,-0.200743,-0.155877,-0.748492,-0.498812,-0.218392,-0.076708,-0.119278,-0.454091,-0.098947,-0.293030,7.960719,0.040562,-0.738175,-0.226642,-0.332141,-0.183496,-0.785238,-2.137560,1.495542,-0.875444,-0.073662,0.047006,1584,1442,1453,PLAG,NTP,NVAX,3.150000,9.420000,277.899994,143.659997,44.606347
1500,-0.103268,-0.190269,-0.145165,-0.002524,-0.196561,1.484821,-0.168588,-0.076708,27.801158,12.162598,0.313806,-0.261994,-0.026214,-0.243739,0.920256,-0.194442,-0.219126,-0.179726,-0.392655,0.329298,0.089388,0.048280,-0.012350,0.051637,1500,263,320,ONEW,BIO,CABO,35.650002,636.830017,2038.500000,1337.665009,36.522159
2016,-0.184181,-0.222114,-0.202208,0.152940,-0.735657,-0.510639,-0.218693,-0.076708,-0.028599,-0.442529,0.232031,-0.268539,-0.184267,-0.252593,-0.119988,-0.227128,-0.353181,-0.183570,-0.824496,-0.428435,0.350896,-0.624460,45.779578,-1.894134,2016,1214,1160,TPHS,LODE,LBRDK,2.370000,5.650000,149.039993,77.344997,31.635021
2227,-0.178376,-0.215553,-0.198259,-0.042117,1.422330,-0.473844,-0.209389,-0.076708,-0.096608,-0.488776,-0.022052,-0.282959,-0.066608,-0.069617,0.232766,-0.220763,-0.321321,-0.170247,2.149320,-0.728463,-0.064793,0.053455,-0.021664,0.051255,2227,805,316,WTRH,FSLY,BYND,3.930000,80.669998,160.220001,120.445000,29.647582
1233,-0.156316,-0.233503,-0.239774,-0.137805,-0.398722,-0.330443,-0.180511,-0.076708,-0.507543,-0.475693,-0.116966,-0.236640,-0.157086,-0.087324,-0.096425,-0.063319,-0.000309,-0.100653,-0.333768,0.280180,-0.706579,-0.595998,-0.056521,0.049416,1233,600,82,LTRPA,DOCU,ALLT,5.260000,264.920013,15.870000,140.395007,25.691065
1214,-0.186754,-0.219454,-0.200808,-0.142766,-0.782186,-0.510475,-0.218686,-0.076708,-0.015439,-0.479953,0.575085,-0.292522,-0.072900,0.049415,-0.456803,-0.227007,-0.342962,-0.182420,-0.822043,-1.871634,0.429349,-0.914256,12.929830,-0.115789,1214,1160,1159,LODE,LBRDK,LBRDA,5.650000,149.039993,145.820007,147.430000,25.093805
23,-0.208300,-0.229212,-0.219299,-0.076836,0.169254,-0.168809,-0.211026,-0.076708,0.591674,0.688704,0.165585,-0.265815,-0.061575,-0.243739,-0.334136,-0.205152,0.113307,-0.172818,0.478389,-0.591433,-0.618865,-0.903906,0.025887,0.048130,23,1259,1969,ACOR,MAXR,TDOC,6.630000,47.820000,292.700012,170.260006,24.680242
1592,-0.221632,-0.220787,-0.221247,0.043028,0.477308,-0.467109,-0.202569,-0.076708,-0.260938,-0.366769,-0.114589,-0.153908,-0.174451,-0.062731,-0.099890,-0.190485,-0.275634,-0.148328,0.284551,-0.423743,0.992137,-0.125080,-0.099135,0.049289,1592,816,2251,PLYA,FVRR,XONE,7.120000,318.769989,44.169998,181.469994,24.487359
1998,0.098253,0.000426,0.063789,-0.000770,0.047315,-0.181457,-0.124976,-0.076708,-0.143275,-0.361596,0.127295,0.137089,-0.084351,-0.277186,0.451072,-0.122595,-0.139174,-0.114807,-0.179188,-1.268763,1.010116,-0.127667,-0.019406,0.052837,1998,919,21,TK,HCHC,ACMR,3.290000,3.990000,123.489998,63.739999,18.373860
31,-0.210298,-0.232170,-0.223325,-0.020034,1.334085,-0.502097,-0.216825,-0.076708,-0.182533,-0.486647,-0.716543,-0.282394,-0.022439,0.482261,-1.359826,-0.228193,-0.324927,-0.173480,1.584982,-0.437508,-0.594893,-0.306202,-0.248649,0.042806,31,1321,805,ADMA,MOSY,FSLY,2.600000,5.640000,80.669998,43.154999,15.598077


## Due Diligence

In [182]:
res[res.Ticker == 'AAPL']

,Levered Free Cash Flow (ttm),EBITDA,Operating Cash Flow (ttm),Total Debt/Equity (mrq),Short P of Float (Jan 28 2021) 4,Revenue Per Share (ttm),Revenue (ttm),Payout Ratio 4,Diluted EPS (ttm),Book Value Per Share (mrq),Return on Equity (ttm),Total Debt (mrq),Quarterly Revenue Growth (yoy),Current Ratio (mrq),Return on Assets (ttm),Gross Profit (ttm),Total Cash Per Share (mrq),Total Cash (mrq),Short P of Shares Outstanding (Jan 28 2021) 4,P Held by Institutions 1,P Held by Insiders 1,Short Ratio (Jan 28 2021) 4,Profit Margin,Operating Margin (ttm),0,1,2,Ticker,Dist_1,Dist_2,Ticker_price,ret_1,ret_2,ret_mean,Dist_from_mean
6,27.31855,22.39444,22.811656,0.007902,-0.756514,-0.229257,13.428127,-0.049715,0.321846,-0.391109,0.575898,11.605968,-0.048739,-0.236853,1.03322,14.192234,-0.087475,14.041313,-0.782784,-0.276388,-0.707124,-0.756421,0.014708,0.052585,6,1335,117,AAPL,MSFT,AMZN,129.850006,241.029999,3251.0,1746.014999,12.446399


In [60]:
# res.to_pickle('res_02_20.pkl')

In [183]:
tickers = ['AAPL','MSFT','AMZN']

In [184]:
raw = pd.read_pickle('combined_data3.pkl')
raw = raw[non_na_cols]
raw[raw.index.isin(tickers)]

Attribute,Levered Free Cash Flow (ttm),EBITDA,Operating Cash Flow (ttm),Total Debt/Equity (mrq),Short P of Float (Jan 28 2021) 4,Revenue Per Share (ttm),Revenue (ttm),Payout Ratio 4,Diluted EPS (ttm),Book Value Per Share (mrq),Return on Equity (ttm),Total Debt (mrq),Quarterly Revenue Growth (yoy),Current Ratio (mrq),Return on Assets (ttm),Gross Profit (ttm),Total Cash Per Share (mrq),Total Cash (mrq),Short P of Shares Outstanding (Jan 28 2021) 4,P Held by Institutions 1,P Held by Insiders 1,Short Ratio (Jan 28 2021) 4,Profit Margin,Operating Margin (ttm)
AAPL,6.689000e+10,8.516000e+10,8.892000e+10,169.19,0.0055,17.13,2.941400e+11,0.2177,3.69,3.94,0.8209,1.120400e+11,0.214,1.16,0.1336,1.049600e+11,4.58,7.683000e+10,0.0055,0.5966,0.0007,0.79,0.2174,0.2525
AMZN,3.664000e+10,4.815000e+10,6.606000e+10,108.38,0.0077,772.13,3.860600e+11,0.0000,41.83,185.69,0.2744,1.012300e+11,0.436,1.05,0.0524,1.527600e+11,167.60,8.440000e+10,0.0066,0.5866,0.1456,0.88,0.0553,0.0593
MSFT,3.479000e+10,7.169000e+10,6.803000e+10,63.56,0.0059,20.23,1.532800e+11,0.3115,6.71,17.26,0.4270,8.278000e+10,0.167,2.58,0.1281,9.694000e+10,17.50,1.319900e+11,0.0059,0.7184,0.0006,1.39,0.3347,0.3924
